In [1]:
%autosave 60 

Autosaving every 60 seconds


In [2]:
#default_exp segmentation.metrics

In [3]:
import fastai; print(fastai.__version__)

1.0.58.dev0


In [4]:
#export
from fastai.vision import *
from fastai.metrics import foreground_acc, dice
from local.test import *

In [5]:
#export
_all_ = ["foreground_acc", "dice"]

In [43]:
a = tensor([[
    [
    [0,0,1],
    [0,0,1],
    [1,1,1]
],
    [
    [1,1,0],
    [1,1,0],
    [0,0,0]
]
]])

b = tensor([[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]])

a.argmax(1)

tensor([[[1, 1, 0],
         [1, 1, 0],
         [0, 0, 0]]])

In [48]:
test_eq(dice(a,b, eps=1e-8), (2*1)/(8))

In [44]:
#export
def iou(input: torch.Tensor, targs: torch.Tensor, **kwargs)->Rank0Tensor:
    "Binary IOU"
    return dice(input, targs, iou=True, **kwargs)

In [45]:
test_eq(iou(a,b, eps=1e-8), 1/7)

In [38]:
#export
def multilabel_dice(input:Tensor, targs:Tensor, c:int, iou:bool=False, mean=True, eps:float=1e-8)->Rank0Tensor:
    "Batch/Dataset Mean Dice"
    input = input.argmax(dim=1, keepdim=True).view(-1)
    targs = targs.view(-1)
    res = []
    for ci in range(c):
        _input, _targs = input == ci, targs == ci
        intersect = (_input * _targs).sum().float()
        union = (_input+_targs).sum().float()
        if not iou: res.append((2. * intersect / union if union > 0 else union.new([1.]).squeeze()))
        else: res.append(intersect / (union-intersect+eps))
    res = torch.tensor(res).to(input.device)
    if not mean: return res
    else: return res.mean()

In [50]:
test_eq(dice(a,b, eps=1e-8), ((2*1)/(8) + (2*2)/(10))/2)

AssertionError: ==:
0.25
0.325

In [20]:
#export
def multilabel_iou(input: torch.Tensor, targs: torch.Tensor, c)->Rank0Tensor:
    "Batch/Dataset Mean IOU"
    return multilabel_dice(input, targs, c=c, iou=True)

In [41]:
test_eq(multilabel_iou(a,b,2), (1/7+2/8)/2)

In [20]:
from local.notebook.export import notebook2script
notebook2script("12_segmentation_metrics.ipynb")

Converted 12_segmentation_metrics.ipynb.
